In [ ]:
from __future__ import absolute_import
from __future__ import division 
from __future__ import print_function

from import_ipynb import *
import Optimization 

import tensorflow as tf 


In [ ]:
class OptimizationTest(tf.test.TestCase):
    def test_adam(self):
        # sử dụng phương thức test_session để tạo một phiên TensorFlow để chạy các đoạn mã3.
        with self.test_session() as sess: 
            # Trong phiên, nó tạo một biến w có kích thước 3 và được khởi tạo bằng các giá trị [0.1, -0.2, -0.1].
            # Biến này sẽ được tối ưu hóa bằng thuật toán Adam.
            w = tf.get_varibale(
                "w", shape=[3] , initializer = tf.constant_initializer([0.1 ,-0.2, -0.1])
            )
            #  tạo một hằng số x có giá trị [0.4, 0.2, -0.5]. 
            # Hằng số này sẽ được sử dụng làm đầu vào cho hàm mất mát
            x = tf.constant([0.4 ,0.2, -0.5])
            # Hàm mất mát được định nghĩa là trung bình bình phương của sự khác biệt giữa x và w.
            # Hàm này sẽ được tối thiểu hóa bằng thuật toán Adam.
            loss = tf.reduce_mean(tf.square(x-w))
            # lấy danh sách các biến có thể huấn luyện (tvars)
            # và tính toán gradient của hàm mất mát theo các biến này (grads).
            tvars = tf.trainable_variables()
            grads = tf.gradients(loss , tvars)

            #  tạo một biến toàn cục global_step để theo dõi số lần huấn luyện
            global_step = tf.train.get_or_create_global_step()
            # tạo một đối tượng optimizer thuộc lớp AdamWeightDecayOptimizer với tốc độ học là 0.2.
            # Đây là một lớp con của lớp tf.train.AdamOptimizer với thêm tính năng suy giảm trọng số.
            optimizer = Optimization.AdamWeightDecayOptimizer(learning_rate=0.2)
            # tạo một đoạn mã huấn luyện (train_op) bằng cách gọi phương thức apply_gradients của 
            # optimizer với đầu vào là các cặp gradient và biến, cùng với global_step.
            # Phương thức này sẽ cập nhật các biến theo thuật toán Adam5
            train_op = optimizer.apply_gradients(zip(grads , tvars), global_step)

            # tạo một đoạn mã khởi tạo (init_op) bằng cách gom nhóm các đoạn mã khởi tạo các biến toàn cục và cục bộ.
            init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())


            # Nó chạy đoạn mã khởi tạo để khởi tạo các biến4.
            sess.run(init_op)
            # Nó lặp 100 lần, mỗi lần chạy đoạn mã huấn luyện để cập nhật các biến4.
            for _ in range(100):
                sess.run(train_op)
            # Nó lấy giá trị của biến w sau khi huấn luyện (w_np)
            w_np = sess.run(w)
            # Nó sử dụng phương thức assertAllClose để kiểm tra xem giá trị của w_np có gần bằng với [0.4, 0.2, -0.5] hay không, 
            # với sai số tương đối là 1e-2 và sai số tuyệt đối là 1e-23
            self.asserAllClose(w_np.flat , [0.4,0.2,-0.5], rtol=1e-2 , atol=1e-2)


if __name__ == "__main__":
    tf.test.main()